In [1]:
# Imports
from utils import read_json_file, write_json_file, read_json_file_or_empty_list
from dotenv import load_dotenv
from typing import  List, Dict
from utils import print_progress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import  math

load_dotenv()

True

In [2]:
# Load coins and markets
from download_chart_data import ChartInterval
from download_chart_data import load_chart_by_interval

coins: List[Dict[str, any]] = read_json_file("data/coins.json")
markets: List[Dict[str, any]] = read_json_file("data/markets.json")
print("\nCoins: {}, Markets: {}".format(len(coins), len(markets)))

# Load daily charts and convert them to daily 
charts = load_chart_by_interval(ChartInterval.FULL)
print("\nCharts with price data count: {}".format(len(charts)))


Coins: 10431, Markets: 10431
 Loading chart_full idx: 10424 / 10425, 100.0%       
Charts with price data count: 9887


In [ ]:
# Find chart pumps
from download_chart_data import chart_data_to_df
from utils import year_month_str

def is_pump(
    price: float, prev_price: float, pump_ratio: float, volume: float, 
    min_volume: float
) -> bool:
    if math.isnan(prev_price) or math.isnan(price):
        return False
    return price >= prev_price * pump_ratio and volume > min_volume


def pumps_info(
    chart_data: List[Dict[str, any]], pump_ratio: float, min_volume: float
) -> pd.DataFrame:
    df = chart_data_to_df(chart_data)
    df['Ym'] = list(map(lambda x: year_month_str(x), df.index))
    # shift price and add pump bool
    df.insert(1, 'prev_price', df['price'].shift(1))
    df['pump'] = df.apply(
        lambda x: is_pump(x['price'], x['prev_price'], pump_ratio, x['volume'], min_volume),
        axis=1
    )
    # select pump rows
    return df.loc[df['pump'] == True]


pump_ratio = 1.5
min_volume = 500000
chart_pumps_df: Dict[str, pd.DataFrame] = dict()

# for each chart
for idx, key in enumerate(charts):
    chart = charts[key]
    print_progress("Searching for pumps", idx, len(charts), True)
    chart_pumps_df[key] = pumps_info(chart, pump_ratio, min_volume)

 Searching for pumps idx: 3915 / 9887, 39.6%       

In [ ]:
# Counting pumps per month
from utils import year_month_keys

keys = year_month_keys(list(range(2017, 2024)), list(range(1, 13)))
df = pd.DataFrame(data={
    'year/month': keys, 
    "pumps": np.zeros(len(keys)), 
    "markets": map(lambda x: "", np.zeros(len(keys)))
})

for idx, key in enumerate(keys):
    for df_key in chart_pumps_df:
        chart_pump_df = chart_pumps_df[df_key]
        pump_cnt = chart_pump_df.loc[chart_pump_df['Ym'] == key].shape[0]
        if pump_cnt > 0:
            df.loc[df['year/month'] == key, 'pumps'] += pump_cnt
            df.loc[df['year/month'] == key, 'markets'] += "{}, ".format(df_key)

    print_progress("Counting for pumps", idx, len(keys), True)
    
chart_pumps_plot_df = df

In [ ]:
df = chart_pumps_plot_df.copy().head(chart_pumps_plot_df.shape[0] - 2).tail(100)
df['legend'] = df.apply(lambda x: x['year/month'][2:], axis=1)
pd.set_option('display.width', 256)
pd.set_option('display.max_colwidth', 80)
print(df.tail(12))
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(df['legend'], df['pumps'])

for i, t in enumerate(ax.get_xticklabels()):
    if (i % 6) != 0:
        t.set_visible(False)

plt.tight_layout() 
plt.show()

In [ ]:
# Display pump for month
key = '2023-10'
row = chart_pumps_plot_df.loc[chart_pumps_plot_df['year/month'] == key]

markets = row['markets'].values[0].split(', ')[:-1]
for market in markets:
    print(market)

In [ ]:
# Generate files and charts for variety of configs

def get_chart_pumps_df(
    charts: Dict[str, Dict[str, any]], pump_ratio: float, min_volume: float
) -> Dict[str, pd.DataFrame]:
    chart_pumps_df: Dict[str, pd.DataFrame] = dict()
    for idx, key in enumerate(charts):
        chart = charts[key]
        print_progress("Searching for pumps", idx, len(charts), True)
        chart_pumps_df[key] = pumps_info(chart, pump_ratio, min_volume)
    return chart_pumps_df


def get_chart_pumps_plot_df(
    keys: List[str], chart_pumps_df: Dict[str, pd.DataFrame]
) -> pd.DataFrame:
    df = pd.DataFrame(data={
        'year/month': keys, 
        "pumps": np.zeros(len(keys)), 
        "markets": map(lambda x: "", np.zeros(len(keys)))
    })
    for idx, key in enumerate(keys):
        for df_key in chart_pumps_df:
            chart_pump_df = chart_pumps_df[df_key]
            pump_cnt = chart_pump_df.loc[chart_pump_df['Ym'] == key].shape[0]
            if pump_cnt > 0:
                df.loc[df['year/month'] == key, 'pumps'] += pump_cnt
                df.loc[df['year/month'] == key, 'markets'] += "{}, ".format(df_key)
    
        print_progress("Counting for pumps", idx, len(keys), True)
    return df


def plot_df(chart_pumps_plot_df: pd.DataFrame):
    df = chart_pumps_plot_df.copy().head(chart_pumps_plot_df.shape[0] - 2).tail(100)
    df['legend'] = df.apply(lambda x: x['year/month'][2:], axis=1)
    print(df.tail(12))
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(df['legend'], df['pumps'])
    for i, t in enumerate(ax.get_xticklabels()):
        if (i % 6) != 0:
            t.set_visible(False)
    plt.tight_layout() 
    plt.show()


def write_pump_to_file(df: pd.DataFrame, pump_ratio: float, min_volume: float):
    results = read_json_file_or_empty_list('results/chart_info.json')
    info = dict()
    pump_count_for_month = dict()
    pump_coins_for_month = dict()
    for key in df['year/month'].values:
        row = chart_pumps_plot_df.loc[chart_pumps_plot_df['year/month'] == key]
        pump_count_for_month[key] = row['pumps'].values[0]
        pump_coins_for_month[key] = row['markets'].values[0].split(', ')[:-1]    
    info['pump_ratio'] = pump_ratio 
    info['min_volume'] = min_volume
    info['pump_count_for_month'] = pump_count_for_month
    info['pump_coins_for_month'] = pump_coins_for_month
    results.append(info)
    write_json_file('results/chart_info.json', results)


pd.set_option('display.width', 256)
pd.set_option('display.max_colwidth', 80)

pump_ratios = [1.5, 2.0, 4.0, 8.0]
min_volumes = [100000, 500000, 1000000, 10000000]
keys = year_month_keys(list(range(2017, 2024)), list(range(1, 13)))

for pump_ratio in pump_ratios:
    for min_volume in min_volumes:
        print("Pump ratio: {}, Min Volume: {}".format(pump_ratio, min_volume))
        print("==============================================================")
        chart_pumps_df = get_chart_pumps_df(charts, pump_ratio, min_volume)
        chart_pumps_plot_df = get_chart_pumps_plot_df(keys, chart_pumps_df)
        plot_df(chart_pumps_plot_df)
        read_json_file_or_empty_list(chart_pumps_plot_df, pump_ratio, min_volume)

pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')